In [7]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date


In [8]:
result = Path('result/')
report = Path('report/')
instances = Path('../../data/csifa')

In [9]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [10]:
MAX_CPU_TIME = 360.0
EPSILON = 1e-6

def clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):

	CSP = (np.zeros((N,N))).tolist()
	CSR = (np.zeros((N,N))).tolist()
	CR = (np.zeros((N,N))).tolist()
	CL = np.zeros(N)

	for i in range(N):
		for j in range(i,N):
			CR[i][j]  = sum(HR[t]*SR[i][t] for t in range(i,j))
			CSP[i][j] = PP[i] * SD[i][j] + sum(HP[t]*SD[t+1][j] for t in range(i,j))
			CSR[i][j] = PR[i] * SD[i][j] + sum(HP[t]*SD[t+1][j] for t in range(i,j))

	for i in range(N):
		CL[i] = sum(HR[j]*SR[i][j] for j in range(i,N))	

	try:
		# create model
		model = gp.Model("clsr_sp_mip")

		# add variables		
		zsp = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="zsp")
		zsr = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="zsr")
		zr = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="zr")
		l = model.addVars(list(range(N)), lb=0.0, ub =1.0, vtype=GRB.CONTINUOUS, name="l")
		yp = model.addVars(list(range(N)), lb=0.0, ub =1.0, vtype=GRB.BINARY, name="yp")
		yr = model.addVars(list(range(N)), lb=0.0, ub =1.0, vtype=GRB.BINARY, name="yr")

		# set objective
		obj = 0.0
		for i in range(N):
			obj += yp[i]*FP[i] + yr[i]*FR[i] + l[i]*CL[i] 
			for j in range(i,N):
				obj += zsp[i,j]*CSP[i][j] + zsr[i,j]*CSR[i][j] + zr[i,j]*CR[i][j] # 16

		model.setObjective(obj, sense = GRB.MINIMIZE)

		# add constraints
		model.addConstr(gp.quicksum(zsp[0,j] + zsr[0,j] for j in range(N)) == 1)
		
		model.addConstrs(
			gp.quicksum(zsp[i,t-1] + zsr[i,t-1] for i in range(t)) - 
			gp.quicksum(zsp[t,j] + zsr[t,j] for j in range(t, N)) == 
			0  for t in range(1,N) 
		)
				
		model.addConstrs(
			gp.quicksum(zsp[t,j] for j in range(t,N)) <= 
			yp[t] for t in range(N)
		)
			
		model.addConstrs(
			gp.quicksum(zsr[t,j] for j in range(t,N)) <= 
			yr[t] for t in range(N)
		)
			
		model.addConstr(gp.quicksum(zr[0,j] for j in range(N)) + l[0] == 1)
					
		model.addConstrs(
			gp.quicksum(zr[i,t-1] for i in range(0,t)) == 
			gp.quicksum(zr[t,j]  for j in  range(t,N)) + l[t] for t in range(1,N)
		)       
				
		model.addConstrs(
			gp.quicksum(zr[i,t] for i in range(0,t+1)) <= 
			yr[t] for t in range(N)
		)    
			
		model.addConstrs(
			gp.quicksum(SR[i][t]*zr[i,t] for i in range(t+1) ) ==
			gp.quicksum(SD[t][j]*zsr[t,j] for j in range(t,N)) for t in range(N)
		)

		model.addConstrs(
		    gp.quicksum(SD[i][t]*zsp[i,t] for t in range(i,N)) <= 
		    yp[i]*min(SD[i][N-1],C) for i in range(N)
		)

		#model.addConstrs(
		#	gp.quicksum(SD[i][t]*zsp[i,t] for t in range(i,N)) <= 
		#	yp[i]*SD[i][N-1] for i in range(N)
		#)
		
		#model.addConstrs(
		#	gp.quicksum(SD[i][t]*zsr[i,t] for t in range(i,N)) <= 
		#	yr[i]*min(SR[0][i],SD[i][N-1],C) for i in range(N)
		#)

		#model.addConstrs(
		#	gp.quicksum(SD[i][t]*zsr[i,t] for t in range(i,N)) <= 
		#	yr[i]*SD[i][N-1] for i in range(N)
		#)
		
		#model.addConstrs(
		#	gp.quicksum(SD[t][k]*zsp[t,k] for k in range(t,N)) + 
		#	gp.quicksum(SD[t][k]*zsr[t,k] for k in range(t,N)) <= 
		#	C for t in range(N)
		#)

		# export .lp
		model.write("clsrp_sp_c52_1.mps")
		
		# parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		model.setParam('OutputFlag', 0)
		#model.setParam('Heuristics', 0)
		model.setParam("LogToConsole", 0)
        
		#logfile = open('callback.log', 'w')

		# optimize model
		model.optimize()

		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

		objval = model.ObjVal
		objbound = model.ObjBound 
		mipgap = model.MIPGap
		runtime = model.Runtime
		nodecount = model.NodeCount 

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return objval, objbound, mipgap, runtime, nodecount, tmp


In [11]:
def main(file_name):
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, tmp = clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_sp_mip.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(bestsol,2))+';'
			   +str(round(bestbound,2))+';'
			   +str(round(gap,2))+';'
			   +str(round(runtime,2))+';'
			   +str(round(numnode,2))+';'
			   +str(round(tmp,2))+'\n')
	arquivo.close()

In [12]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,2):
			datafile = f"c{dim}_{id}.txt"
			print(f"Resolvendo instancia {datafile}")			
			main(datafile)

Resolvendo instancia c52_1.txt
Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-19
Set parameter TimeLimit to value 360
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1


## Table of results

In [ ]:
data=f'result/clsr_sp_mip.csv'

df = pd.DataFrame()
df = pd.read_csv(data,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','objval','objbound','mipgap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'objval':tab["objval"].mean(),
    'objbound':tab["objbound"].mean(),
    'mipgap':tab['mipgap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"uls_mip"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["objval"] = tab["objval"].round(2)
tab["objbound"] = tab["objbound"].round(2)
tab["mipgap"] = tab["mipgap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')



In [ ]:
tab

In [ ]:
#tab
print(
    tab[['instance','objval','objbound','mipgap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)